#Suburban neighborhood design

##Examples and reference model

<a href="http://lyonvistaverde.com/beta2/i/VistaVerde_Site%20Plan%20Only_R_07-01-11-xl.jpg">
<img src="img/model.jpg" style="width: 500px; height: 400px">
</a>

##Previous Workshops modification

To accomplish the goals of this workshop some slight modification were made, in order to create a more realistic scene with different elements, the previous workshop was modified inserting a random mechanism that generate almost every important part of the scene in a random way.

Thanks to this modifications the main function can create theoretically thousands of different flavour of the two base house models (at least one must be pretty :wink:).

###workshop_08 (house spatial frame)
> Randomization of different house parts, such as roof, exteriors, roof-top and balconies.

###workshop_10 (multistorey house)
> Added a small wrapper to the main function and some randomization, now the generated house type can be choosen.

##Extra modules

In order to add some green to the scene, an extra modules [tree.py](src/tree.py) was written, this module allow the construction of a very simple tree model starting from a base one, the resulting model is "randomized" in height, width and color (shades of green), with respect to the needed proportion reuested by the scene.

##Variables used in the workshop

In this workshops the variables used were nearly the same of the previuos workshop and that's quite natural, by the way a brief reminder is given, for further clarification see [workshop_10](https://github.com/gabmarini/ggpl/blob/master/2017-01-13/workshop_10.ipynb), the new files/variables are **highlighted**: 

###walls, doors and windows

* muriesterniX.lines
* muriinterniX.lines
* terrace_wallsX.lines
* porte_modelX.lines
* finestre_modelX.lines

###surfaces

* livingroomXY/bathroomXY/bedroomXY/terrace_surfaceXY.lines
* **bezierX.lines**
> used to build roads and sidewalks

###ladder/stair

* ladder.lines

###trees

* **trees.lines**
> used to position trees

##Helper functions

```python
def box_max_min(boxVerts):
	"""
	box_max_min is a function that given a list of vertices, return two vertices, the ones
	visible and not covered by the plastic's borders. The former is the vertex with
	minimum X and Y values, meanwhile the latter is the one with maximum X and Y
	values, together they describe a 2D box used to put in place the trees.
	@param boxVerts: a list of vertices
	@return p1,p2: two points describing a 2D box in XY plane (z = 0)
	"""
        #implementation here...

def is_in_box(boxMinV, boxMaxV, point):
	"""
	is_in_box is a function that given a 2 points describing a box in XY plane, return True
	if the third point passed as parameter is contained inside the box, False otherwise.
	@param boxMinV: the vertex with	minimum X and Y values 
	@param the vertex with maximum X and Y values
	@param point: is the vertex to be verified
	@return boolean: True if the point is contained in the box, False otherwise
	"""
        #implementation here...


def build_district(n, squared = False):
	"""
	build_district is a function that given a number and an optional boolean parameter,
	return a district of N houses if squared is false (or not passed).
	If the second parameter is not False, 4 houses will be produced, arranged in 
	a square, one house for every vertex of the square.
	@param n: number of houses that need to be built.
	@param squared: if the houses need to be arranged as a square,
	actually override the first parameter to 4.
	@return district: the HPC model of the district
	"""
        #implementation here...

def position_trees(box):
	"""
	position_trees is a function that given an hpc model of a box on XY plane reads 
	a .lines file and return a list of HPC models representing a bunch of trees. 
	A tree is created at the beginning and the end of every line in the .lines file. 
	A tree isn't created if it will result outside the box or on the plastic border. 
	The trees are slightly randomized in shape and colors.
	@param box: the box in which the trees will be positioned.
	@return trees: a list of tree hpc models
	"""
        #implementation here...

def chunks(l, n):
    """Yield successive n+1-sized chunks from l."""
        #implementation here...

def draw_beziers():
	"""
	draw_beziers is a function that reads a bunch of .lines file and generate the 
	related bezier's curves, every curve is built 2 time, the former is 
	used in order to draw the road, meanwhile the latter is used for the sidewalk.
	@return roads: the HPC model of the roads plus the sidewalks
	"""
        #implementation here...
```

##Main function

```python
def suburban_neighborhood():
	"""
	suburban_neighborhood is a function that generate an HPC model of the entire 
	neighborhood, although it won't take any argument it's capable of generating a 
	different scenario every time it is launched. Due to the randomization used 
	it can generate hundreds of different trees and theoretically thousands of 
	different house (starting from two different house's base type).
	It generate districts, roads, sidewalks, trees, rivers and the plastic box
	and borders.
	@return neighborhood: the HPC model of the neighborhood.
	"""

	#building roads and sidewalks
	roads = draw_beziers()

	#building box and border
	box = BOX([1,2])(roads)
	box = MATERIAL([0,0,0,1,  0,.1,0,1,  0,.1,0,1, 0,0,0,1, 0])(box)
	border = PROD([T([1,2])([-1,-1])(OFFSET([2,2])(SKEL_1(box))),Q(2)])
	border = MATERIAL([0.53,.3,0,.1,  0,0,0,1,  0.53,.3,0,.8, 0,0,0,1, 100])(border)

	#building rivers
	boxMinV,boxMaxV = box_max_min(UKPOL((JOIN(SKEL_1(border))))[0])

	points = [[[boxMaxV[0],boxMinV[1]], [boxMaxV[0],boxMinV[1]+.75*i], [boxMaxV[0]-2.5*i,boxMinV[1]]] for i in xrange(1,32)]
	points2 = [[[boxMinV[0],boxMaxV[1]], [boxMinV[0],boxMaxV[1]-.75*i], [boxMinV[0]+.85*i,boxMaxV[1]]] for i in xrange(1,45)]

	rivers = [MKPOL([ps,[[1,2,3]],1]) for ps in points]
	rivers2 = [MKPOL([ps,[[1,2,3]],1]) for ps in points2]

	riversModel = [waterify(T([3])([.005*i+1])(fiume)) for i,fiume in enumerate(rivers,0)]
	riversModel2 = [waterify(T([3])([.005*i+1])(fiume)) for i,fiume in enumerate(rivers2,0)]

	#assebling the base
	base = S([1,2])([1.45,1.45])(STRUCT([box, roads, border, STRUCT(riversModel + riversModel2)]))

	#building every single district
	district = R([1,2])(-PI/2.44)(build_district(6))
	district2 = R([1,2])(PI/10)(build_district(3))
	district3 = R([1,2])(PI/2.5)(build_district(3))
	district4 = R([1,2])(-PI+PI/5.)(build_district(5))
	district5 = R([1,2])(PI/5.)(build_district(5))
	district6 = R([1,2])(-PI+PI/5.)(build_district(4))
	district7 = R([1,2])(PI/5.)(build_district(5))
	district8 = R([1,2])(PI/8.)(build_district(0,True))
	district9 = R([1,2])(-PI/2.60)(build_district(3))
	district10 = R([1,2])(-PI/2.44)(build_district(6))
	district11 = R([1,2])(-PI/3.5)(build_district(3))

	#building the entire neighborhood
	model = STRUCT([base, T([1,2])([150,262])(district), 
	T([1,2])([245,300])(district2), 
	T([1,2])([204,353])(district3), 
	T([1,2])([97,366])(district4),
	T([1,2])([170,285])(district5),
	T([1,2])([53,330])(district6),
	T([1,2])([120,257])(district7),
	T([1,2])([193,285])(district8),
	T([1,2])([80,235])(district9),
	T([1,2])([168,217])(district10),
	T([1,2])([38,362])(district11)])

	#building trees
	trees = position_trees(box)

	#assembling all together
	neighborhood = STRUCT([model, trees])
	
	#translating back to the origin	
	boxMinV,boxMaxV = box_max_min(UKPOL(S([1,2])([1.45,1.45])(JOIN(SKEL_1(border))))[0])
	neighborhood = T([1,2])([-boxMinV[0], -boxMinV[1]])(neighborhood)

	#returning the result
	return neighborhood
```

##Results

Below are reported various stage of the execution that might be a bit time consuming, i.e. on low-medium spec machines the entire computation and rendering of the scene can take up to ~15min, that's due to the calculus made in order to produce a single house (every house is completely built both inside and outside).

###Rivers

The rivers were realized as a bunch of semi-transaparent overlapped laminae, the smaller is positioned in the left corner, the other tend to grow bigger (though mantaining the left corner fixed on that particular point) resulting in a fade-away effect to the right. Moreover the water try to reflect the light as realistic as possible.

<table>
 <tbody>
    <tr>
        <td><img src="img/water1.png" style="width: 500px; height: 400px"></td>
        <td><img src="img/water2.png" style="width: 100px; height: 400px"></td>
        <td><img src="img/water3.png" style="width: 100px; height: 400px"></td>
    </tr>
</tbody>
</table>

###The base (box and borders)

The base was realized as a dark green box with a sorrounding light brown frame, both the colors of the box and the frame were realized used the MATERIAL primitive, in fact they tend to reflect the light in a different way, as can be seen below.
<table>
 <tbody>
    <tr>
        <td><img src="img/base1.png" style="width: 500px; height: 400px"></td>
        <td><img src="img/base2.png" style="width: 500px; height: 400px"></td>
    </tr>
</tbody>
</table>

###Testing the final result

Due to the low computational power available, all the experiments about the houses's positioning were made using a test function, in this way there's no need to render the entire house but only its box, that can suffice for the testing purpose, and moreover the render of the scene is faster, but the final results can be seen quite clearly.
<table>
 <tbody>
    <tr>
        <td><img src="img/test1.png" style="width: 700px; height: 300px"></td>
    </tr>
    <tr>
        <td><img src="img/test2.png" style="width: 700px; height: 500px"></td>
    </tr>
</tbody>
</table>

###Final result

<table>
 <tbody>
    <tr>
        <td><img src="img/fine1.png" style="width: 700px; height: 250px"></td>
    </tr>
    
    <tr>
        <td><img src="img/fine3.png" style="width: 700px; height: 150px"></td>
    </tr>
    
    <tr>
        <td><img src="img/fine2.png" style="width: 700px; height: 550px"></td>
    </tr>
</tbody>
</table>

###Some particular of the scene

<table>
 <tbody>
    <tr>
        <td><img src="img/part1.png" style="width: 700px; height: 400px"></td>
    </tr>
    
    <tr>
        <td><img src="img/part2.png" style="width: 700px; height: 400px"></td>
    </tr>
</tbody>
</table>